# VacationPy

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [18]:
clean_city_data = pd.read_csv("../output_data/cities.csv")
clean_city_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ojinaga,29.5667,-104.4167,71.60,16,1,3.44,MX,1612563006
1,Provideniya,64.3833,-173.3000,7.27,90,100,14.41,RU,1612562552
2,Porto-Novo,6.4965,2.6036,86.00,79,40,12.66,BJ,1612562619
3,Marawi,7.9986,124.2928,71.01,91,89,7.00,PH,1612563007
4,Iqaluit,63.7506,-68.5145,23.00,80,75,18.41,CA,1612562549
...,...,...,...,...,...,...,...,...,...
542,Gravdal,68.1214,13.5031,30.20,86,40,6.91,NO,1612563137
543,Tual,-5.6667,132.7500,83.62,75,99,20.15,ID,1612562637
544,Schwarzenberg,50.5500,12.7833,33.01,95,100,3.00,DE,1612563138
545,Mnogovershinnyy,53.9353,139.9242,-14.62,91,100,4.03,RU,1612563138


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [22]:
# configure gmaps with API key
gmaps.configure(api_key=g_key)

# setting locations and weight
clean_city_data = clean_city_data.dropna()

locations = clean_city_data[["Lat", "Lng"]].astype(float)
humidity = clean_city_data["Humidity"].astype(float)

# Create a heatmap layer
fig = gmaps.figure(zoom_level=1.5, center=(20,20))

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 max_intensity=max(clean_city_data["Humidity"]))
    
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [20]:
# create my ideal weather conditions: max temperature < 85 degrees but > 70,
# humidity between 30-50%, and low change of cloudiness.

ideal_weather_df = clean_city_data.loc[(clean_city_data["Max Temp"] >= 70) & 
                                        (clean_city_data["Max Temp"] <= 85) & 
                                        (clean_city_data["Humidity"] >= 30) &
                                        (clean_city_data["Humidity"] <= 50) & 
                                        (clean_city_data["Cloudiness"] <= 20)]

# drop any rows will null values
ideal_weather_df = ideal_weather_df.dropna()    

# display dataframe
ideal_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
79,Ciudad Río Bravo,25.9833,-98.1000,73.40,43,20,12.66,MX,1612563025
140,Formosa,-26.1775,-58.1781,82.40,47,0,10.36,AR,1612563041
154,Ancud,-41.8697,-73.8203,82.40,32,0,16.11,CL,1612562650
187,Orodara,10.9492,-4.9342,80.60,36,10,8.05,BF,1612563051
265,Doctor Phillips,28.4495,-81.4923,78.80,41,1,11.50,US,1612563071
281,Mandera,3.9366,41.8670,80.37,30,2,7.36,KE,1612563075
329,São João da Barra,-21.6403,-41.0511,82.40,32,0,11.50,BR,1612562578
382,Coyhaique,-45.5752,-72.0662,84.20,30,0,6.91,CL,1612562808
396,Cauquenes,-35.9671,-72.3225,78.15,42,0,6.55,CL,1612563102
520,Red Bluff,40.1785,-122.2358,71.60,36,1,3.44,US,1612563066


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels within 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
# create new hotel dataframe
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng"]].copy()

# add a new column for Hotel Name and set empty parameters to store data
hotel_df["Hotel Name"] = ""

# set parameters to search for hotels within 5000 meters
parameters = {
    "type": "lodging",
    "radius": 5000,
    "key": g_key,
}

for index, row in hotel_df.iterrows():
    # set parameters to loop through each city's coordinates
    lat = row["Lat"]
    lng = row["Lng"]
    parameters["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make API request
    response = requests.get(base_url, params=parameters).json()
    
    # extract search results
    results = response["results"]
    
    # store search results to Hotel Name column
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    except:
        print("no hotel found")
    
hotel_df

,City,Country,Lat,Lng,Hotel Name
79,Ciudad Río Bravo,MX,25.9833,-98.1000,Villa Santa Cecilia
140,Formosa,AR,-26.1775,-58.1781,Asterión Hotel
154,Ancud,CL,-41.8697,-73.8203,Hotel Arena Gruesa
187,Orodara,BF,10.9492,-4.9342,Bkf Orodara
265,Doctor Phillips,US,28.4495,-81.4923,Drury Inn & Suites near Universal Orlando Resort™
281,Mandera,KE,3.9366,41.8670,Kornesh Hotel and Lodging
329,São João da Barra,BR,-21.6403,-41.0511,Pousada Porto De Canoas
382,Coyhaique,CL,-45.5752,-72.0662,Diego De Almagro Coyhaique
396,Cauquenes,CL,-35.9671,-72.3225,Viña Don Heraldo
520,Red Bluff,US,40.1785,-122.2358,Hampton Inn & Suites Red Bluff


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name:</dt><dd>{Hotel Name}</dd>
<dt>City:</dt><dd>{City}</dd>
<dt>Country:</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
pin_locations = hotel_df[["Lat", "Lng"]]

In [21]:
# Plot hotels on top of the humidity heatmap with pins containing Hotel Name, City, and Country.

# Create a marker_layer using the hotel list to fill the info box
fig = gmaps.figure(zoom_level=1.5, center=(20,20))
markers = gmaps.marker_layer(pin_locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display figure with hotel names on top of heat map
fig

Figure(layout=FigureLayout(height='420px'))